In [1]:

print("helloworld")



helloworld


In [1]:

import os
import findspark
findspark.init('/Users/jerdavis/devlib/spark-2.4.3-bin-hadoop2.7/')

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /Users/jerdavis/devlib/xgboost/xgboost4j-spark-0.90.jar,/Users/jerdavis/devlib/xgboost/xgboost4j-0.90.jar,/Users/jerdavis/devhome/projects/pysparkgw/target/scala-2.11/pysparkgw_2.11-0.1.jar pyspark-shell'
findspark.init()


In [2]:


import numpy
import pyspark

from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("foo")\
    .master("local[*]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G")\
    .getOrCreate()

    # .config("spark.driver.extraClassPath", "/Users/jerdavis/devlib/xgboost/xgboost4j-spark-0.90.jar:/Users/jerdavis/devlib/xgboost/xgboost4j-0.90.jar:/Users/jerdavis/devhome/projects/pysparkgw/target/scala-2.11/pysparkgw_2.11-0.1.jar")\
    # .config("spark.executor.extraClassPath", "/Users/jerdavis/devlib/xgboost/xgboost4j-spark-0.90.jar:/Users/jerdavis/devlib/xgboost/xgboost4j-0.90.jar:/Users/jerdavis/devhome/projects/pysparkgw/target/scala-2.11/pysparkgw_2.11-0.1.jar")\
    # .config("spark.jars", "/Users/jerdavis/devlib/xgboost/xgboost4j-spark-0.90.jar:/Users/jerdavis/devlib/xgboost/xgboost4j-0.90.jar:/Users/jerdavis/devhome/projects/pysparkgw/target/scala-2.11/pysparkgw_2.11-0.1.jar")\

#spark.sparkContext.addPyFile("/Users/jerdavis/devlib/xgboost/sparkxgb.zip")

#import sparkxgb
#from sparkxgb import XGBoostEstimator


# selfHelper = spark.sparkContext._jvm.jd.PyTest
# selfHelper.quoteRandallzz()



In [3]:

selfHelper = spark.sparkContext._jvm.jd.PyTest
#selfHelper.foo(123,"smoo")
print(selfHelper.bar(123,"smoo",spark.sparkContext._jsc))

rr = spark.sql("SELECT * from rr")
rr.show()

# schema = StructType(
#   [StructField("PassengerId", DoubleType()),
#     StructField("Survival", DoubleType()),
#     StructField("Pclass", DoubleType()),
#     StructField("Name", StringType()),
#     StructField("Sex", StringType()),
#     StructField("Age", DoubleType()),
#     StructField("SibSp", DoubleType()),
#     StructField("Parch", DoubleType()),
#     StructField("Ticket", StringType()),
#     StructField("Fare", DoubleType()),
#     StructField("Cabin", StringType()),
#     StructField("Embarked", StringType())
#   ])
# 
# df_raw = spark\
#   .read\
#   .option("header", "true")\
#   .schema(schema)\
#   .csv("/Users/jerdavis/devlib/xgboost/train.csv")
# df = df_raw.na.fill(0)



+-----------+----------+
|PassengerId|prediction|
+-----------+----------+
|        1.0|       0.0|
|        4.0|       1.0|
|       14.0|       0.0|
|       15.0|       1.0|
|       20.0|       1.0|
|       28.0|       1.0|
|       34.0|       0.0|
|       38.0|       0.0|
|       50.0|       1.0|
|       52.0|       0.0|
|       59.0|       1.0|
|       60.0|       0.0|
|       82.0|       0.0|
|       94.0|       0.0|
|       96.0|       0.0|
|       99.0|       1.0|
|      104.0|       0.0|
|      105.0|       0.0|
|      107.0|       1.0|
|      116.0|       0.0|
+-----------+----------+
only showing top 20 rows



In [10]:
sexIndexer = StringIndexer()\
  .setInputCol("Sex")\
  .setOutputCol("SexIndex")\
  .setHandleInvalid("keep")
    
cabinIndexer = StringIndexer()\
  .setInputCol("Cabin")\
  .setOutputCol("CabinIndex")\
  .setHandleInvalid("keep")
    
embarkedIndexer = StringIndexer()\
  .setInputCol("Embarked")\
  .setOutputCol("EmbarkedIndex")\
  .setHandleInvalid("keep")

vectorAssembler = VectorAssembler()\
  .setInputCols(["Pclass", "SexIndex", "Age", "SibSp", "Parch", "Fare", "CabinIndex", "EmbarkedIndex"])\
  .setOutputCol("features")

foo = {"featuresCol":"features","labelCol":"Survival","predictionCol":"prediction"}

#xgboost = XGBoostEstimator(foo)
    # featuresCol="features", 
    # labelCol="Survival", 
    # predictionCol="prediction",
    # maxDepth="20",


# xgboost = XGBoostEstimator(
#     featuresCol="features", 
#     labelCol="Survival", 
#     predictionCol="prediction")
    
 
pipeline = Pipeline().setStages([sexIndexer, cabinIndexer, embarkedIndexer, vectorAssembler, xgboost])



In [5]:
trainDF, testDF = df.randomSplit([0.8, 0.2], seed=24)
model = pipeline.fit(trainDF)
model.transform(testDF).select(col("PassengerId"), col("prediction")).show()


+-----------+----------+
|PassengerId|prediction|
+-----------+----------+
|        1.0|       0.0|
|        4.0|       1.0|
|       14.0|       0.0|
|       15.0|       1.0|
|       20.0|       1.0|
|       28.0|       1.0|
|       34.0|       0.0|
|       38.0|       0.0|
|       50.0|       1.0|
|       52.0|       0.0|
|       59.0|       1.0|
|       60.0|       0.0|
|       82.0|       0.0|
|       94.0|       0.0|
|       96.0|       0.0|
|       99.0|       1.0|
|      104.0|       0.0|
|      105.0|       0.0|
|      107.0|       1.0|
|      116.0|       0.0|
+-----------+----------+
only showing top 20 rows



In [4]:


def train_help(args):
 
   try:
      xgbWrapper = spark.sparkContext._jvm.jd.PyTest
      
      best_score = xgbWrapper.eval(spark.sparkContext._jsc,
                                   '/Users/jerdavis/devlib/xgboost/train.csv', 
                                   ["Pclass", "SexIndex", "Age", "SibSp", "Parch", "Fare", "CabinIndex", "EmbarkedIndex"],
                                   'Survival',
                                   float(args['params']['eta']), 
                                   float(args['params']['gamma']),
                                   int(args['params']['max_depth']),
                                   int(args['params']['max_leaves']),
                                   float(args['params']['min_child_weight']),
                                   float(args['params']['subsample']),
                                   float(args['params']['colsample_bytree']),
                                   float(args['params']['colsample_bylevel']),
                                   float(args['params']['colsample_bynode']),
                                   float(args['params']['lambda']),
                                   float(args['params']['alpha']),
                                   int(args['params']['max_bin']),
                                   int(args['params']['num_parallel_tree']),
                                   int(args['params']['num_round']),
                                   int(args['params']['num_early_stopping_rounds']))

      
      return 1.0 - best_score
   except Exception as e:
       print(str(e))
       return float(1)
 

from hyperopt import tpe, hp, fmin, Trials

param_space = hp.choice( 'foo', [{'model':'foom', 'params': {'eta':hp.uniform('eta', 0, 1),
                                                             'gamma':1.0,
                                                             'max_depth':hp.quniform('max_depth', 3, 60, 3),
                                                             'max_leaves':0,
                                                             'min_child_weight':1.0,
                                                             'subsample':1.0,
                                                             'colsample_bytree':1.0,
                                                             'colsample_bylevel':1.0,
                                                             'colsample_bynode':1.0,
                                                             'lambda':1.0,
                                                             'alpha':0.0,
                                                             'max_bin':hp.quniform('max_bin', 32, 512, 32),
                                                             'num_parallel_tree':hp.quniform('num_parallel_tree', 2, 60, 2),
                                                             'num_round':100,
                                                             'num_early_stopping_rounds':10,
                                                             }}])

trials = Trials()
best_params = fmin( fn=train_help, space = param_space, algo=tpe.suggest, max_evals = 120, trials = trials)
print(best_params)



100%|██████████| 120/120 [03:12<00:00,  1.50s/it, best loss: 0.00030499696731567383]
{'eta': 0.9113941275169696, 'foo': 0, 'max_bin': 96.0, 'max_depth': 39.0, 'num_parallel_tree': 4.0}


In [ ]:
import pickle
from hyperopt import tpe, hp, fmin, Trials
param_space = hp.choice( 'foo', [{'model':'foom', 'params': {'eta':hp.uniform('eta', 0, 1),
                                                             'gamma':1.0,
                                                             'max_depth':hp.quniform('max_depth', 3, 60, 3),
                                                             'max_leaves':0,
                                                             'min_child_weight':1.0,
                                                             'subsample':1.0,
                                                             'colsample_bytree':1.0,
                                                             'colsample_bylevel':1.0,
                                                             'colsample_bynode':1.0,
                                                             'lambda':1.0,
                                                             'alpha':0.0,
                                                             'max_bin':hp.quniform('max_bin', 32, 512, 32),
                                                             'num_parallel_tree':hp.quniform('num_parallel_tree', 2, 60, 2),
                                                             'num_round':100,
                                                             'num_early_stopping_rounds':10,
                                                             }}])


def run_trials():

    trials_step = 1  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 2  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("/tmp/my_model.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    #best = fmin(fn=mymodel, space=model_space, algo=tpe.suggest, max_evals=max_trials, trials=trials)
    best_params = fmin( fn=train_help, space = param_space, algo=tpe.suggest, max_evals=max_trials, trials = trials)

    print("Best:", best_params)
    
    # save the trials object
    with open("/tmp/my_model.hyperopt", "wb") as f:
        pickle.dump(trials, f)

# loop indefinitely and stop whenever you like
while True:
    run_trials()
    


100%|██████████| 5/5 [00:08<00:00,  1.60s/it, best loss: 0.0010610222816467285]
Best: {'eta': 0.9839068958201556, 'foo': 0, 'max_bin': 224.0, 'max_depth': 45.0, 'num_parallel_tree': 14.0}
Found saved Trials! Loading...
Rerunning from 5 trials to 8 (+3) trials
100%|██████████| 3/3 [00:04<00:00,  1.51s/it, best loss: 0.0010610222816467285]
Best: {'eta': 0.9839068958201556, 'foo': 0, 'max_bin': 224.0, 'max_depth': 45.0, 'num_parallel_tree': 14.0}
Found saved Trials! Loading...
Rerunning from 8 trials to 11 (+3) trials
100%|██████████| 3/3 [00:04<00:00,  1.55s/it, best loss: 0.0010610222816467285]
Best: {'eta': 0.9839068958201556, 'foo': 0, 'max_bin': 224.0, 'max_depth': 45.0, 'num_parallel_tree': 14.0}
Found saved Trials! Loading...
Rerunning from 11 trials to 14 (+3) trials
100%|██████████| 3/3 [00:05<00:00,  1.64s/it, best loss: 0.0010610222816467285]
Best: {'eta': 0.9839068958201556, 'foo': 0, 'max_bin': 224.0, 'max_depth': 45.0, 'num_parallel_tree': 14.0}
Found saved Trials! Loading..